# Synchrotron Radiation: Cooling Time and Power Laws

## Pre-Requisites
As usual, we start with importing the necessary modules.

In [ ]:
from astropy import units as u
from astropy import constants as c

from astropy.modeling.powerlaws import PowerLaw1D

import numpy as np

import matplotlib.pyplot as plt

from math import pi
from numpy import sqrt, cos, sin, tan, abs

## Synchrotron Cooling Time

The synchrotron cooling time (lecture, slide 54) is
$$
\tau_\mathsf{sync} (U) = \frac{3}{2} \frac{c m_e^2}{\varepsilon_0 \sigma_T \beta B^2 U}
$$
where $B$ is the magnetic field and $U$ is the total energy of the electron. We will be dealing only with electrons so we will use the eletron mass for the particle's mass.
### Exercise
Write a function that calculates the cooling time for a given magnetic field and (an array of) electron energy.

In [ ]:
def CoolingTime_Sync (U, B = 5e-8*u.T, beta = 1.) :
    
    # your code here
    tau = U.value * 0 * u.s
    
    return tau.to(u.s)

Let's test the function for an array of four electron energies:

In [ ]:
U = [1e-1, 1, 10] * u.TeV

In [ ]:
CoolingTime_Sync (U)

Make a plot of the cooling time vs. electron energy for electron energies between 1 MeV and 100 TeV.

In [ ]:
# your code here

## Synchrotron Radiation from an Electron Population
So far we have dealt with mono-energetic electrons. We usually observe electrons of many different energies, often they follow a power law. Let's define our power law as in the Introduction.

In [ ]:
E_0 = 1 * u.TeV
A = 10 / u.TeV
Gamma = 2.5

In [ ]:
PL = PowerLaw1D(A, E_0, Gamma)

We want to take a look at the synchrotron spectrum. So we need our functions for the synchrotron spectrum from the last notebook:

In [ ]:
import scipy.integrate as integrate
import scipy.special as special

def F(x) :
    ## https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html
    
    #K53 = special.kv(5./3, x)
    result = x*integrate.quad(lambda z: special.kv(5./3,z), x, np.inf)[0]
    
    return result

def F_vec(x_vec) :
    
    integrateArray = []
    
    for x in x_vec :
        integrate = F(x)
        integrateArray.append(integrate)
    
    return integrateArray    

def nu_sync (U, B, phi = 90*u.degree) : 
    
    ret = U**2 * c.e.si * B * sin(phi) / 2 / pi / c.m_e**3 / c.c**4
    
    return ret.decompose().to(u.Hz)

def Psync(nu, E, B, phi = 90*u.degree) : 

    nu_over_nus = nu / nu_sync (E, B, phi)
    
    nu_over_nus = nu_over_nus.decompose().value
    
    if type(nu_over_nus) is np.ndarray :
        #print('array mode')
        ret = np.array(F_vec(nu_over_nus))*u.one
    else :
        #print('single mode')
        ret = F(nu_over_nus)*u.one


    ret *= sqrt(3)/8/pi**2 
    
    ret *= c.e.si**3 * B * sin(phi) / c.m_e / c.c / c.eps0.si


    return ret.to(u.W/u.Hz)


We define a range of photon frequencies:

In [ ]:
frequencies = np.logspace(13, 19, 601) * u.Hz

And we use our magnetic field of $B = 10^{-8}\,\mathsf{T}$:

In [ ]:
B = 1e-8 * u.T

Let's get the synchrotron spectra for electron energies of 100 GeV, 1 TeV and 10 TeV: 

In [ ]:
spec_100GeV = Psync(frequencies, 100*u.GeV, B)
spec_1TeV = Psync(frequencies, 1*u.TeV, B)
spec_10TeV = Psync(frequencies, 10*u.TeV, B)

But we have more electrons at 1 GeV then at 10 GeV. How many electrons? This comes from the power law defined above. We have to multiply the spectrum with the number of electrons:

In [ ]:
spec_100GeV *= PL(100*u.GeV)
spec_1TeV *= PL(1*u.TeV)
spec_10TeV *= PL(10*u.TeV)

We have to sum these spectra to get the total emission:

In [ ]:
spec_sum = spec_100GeV.copy()

spec_sum += spec_1TeV
spec_sum += spec_10TeV

In [ ]:
plt.loglog(frequencies, spec_sum, label = "total emission", linewidth = 4)

plt.loglog(frequencies, spec_100GeV, label = "100 GeV")
plt.loglog(frequencies, spec_1TeV, label = "1 TeV")
plt.loglog(frequencies, spec_10TeV, label = "10 TeV")

plt.legend()

#p we need to limit the y range a bit:
plt.ylim(1e-40,1e-30)

plt.xlabel('frequency [{}]'.format(frequencies.unit))
plt.ylabel('P [{}]'.format(spec_sum.unit))

### Exercise
Keep the electron energy range from 100 GeV to 10 TeV. Make the same plot as above but for many more intermediate energies. Create an array of electron energies and loop over it. You should have enough intermediate energies that the total emission is almost a power law.

Take the part of the spectrum following clearly a power law. Find out the spectral index. You can do that by taking two values and doing the calculation on a sheet of paper. You can also do a least-square fit of the total emission..

In [ ]:
# Here is an empty spectrum to be filled in

spec_sum = np.zeros(len(frequencies)) * u.W / (u.Hz * u.TeV)

In [ ]:
#exec(open('PowerLaws_solution.py').read())